In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np
import pandas

#GUI elements
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%run logic.ipynb
%run participant.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
#Example JSON
root_question = {'qid':0,
'question': 'Blood type?',
'answers': ['O+','A+','Other'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Specify...',
'answers': ['B+','AB+','O-','A-','Other'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

follow_up_2 = {'qid':2,
'question': 'Specify further...',
'answers': ['B-','AB-'],
'probability': ['1/2','1/2']}

paths = [[0,'Other',1], [1,'Other',2]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1, follow_up_2],
    'paths':paths,
    'truth':['1/4']
}
json_poll

{'children': [{'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
   'probability': ['1/5', '1/5', '1/5', '1/5', '1/5'],
   'qid': 1,
   'question': 'Specify...'},
  {'answers': ['B-', 'AB-'],
   'probability': ['1/2', '1/2'],
   'qid': 2,
   'question': 'Specify further...'}],
 'paths': [[0, 'Other', 1], [1, 'Other', 2]],
 'roots': [{'answers': ['O+', 'A+', 'Other'],
   'probability': ['1/3', '1/3', '1/3'],
   'qid': 0,
   'question': 'Blood type?'}],
 'truth': ['1/4']}

In [4]:
%run gui_components.ipynb

<IPython.core.display.Javascript object>

In [5]:
# Distribution sliders per question (non-flattened)
children = []

subtrees = pollToSubtrees(json_poll)
for question in subtrees:
    for subquestion in subtrees[question]:
        html = widgets.HTML('<b>Question:</b> '+subquestion['question'])
        to_add = []
        
        count = 0
        for answer in subquestion['answers']:
            probability = subquestion['probability'][count]
            to_add.append(widgets.Label(value=answer))
            to_add.append(widgets.FloatSlider(
                value=probability,
                min=0,
                max=1.0,
                step=0.01,
                description='Weight:',
                disabled=False,
                continuous_update=False,
                orientation='vertical',
                readout=True,
                readout_format='.1f',
            ))
            count +=1
        
        contents =[html, widgets.HBox(to_add)]
        children.append( widgets.VBox(contents))
        
dist_accordion.children = children
dist_box = widgets.VBox([dist_accordion])

In [6]:
display(start_tab)

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0lucHV0IEpTT046JyksIFNlbGVjdChvcHRpb25zPSgnYmxvb2QuanNvbicsKSwgdmFsdWXigKY=


In [7]:
display(widgets.HTML('<center><h1> Input Distribution</h1></center>'))
display(dist_accordion)

HTML(value=u'<center><h1> Input Distribution</h1></center>')

QWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGI+UXVlc3Rpb246PC9iPiBCbG9vZCB0eXBlPycpLCBIQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0/igKY=


In [8]:
participants = []
n = 100000
participant_budget = 40

matrices = pollToMatrix(json_poll)
epsilons = pollToEpsilon(json_poll)

#TODO: Loop x times for confidence

for i in range(0, n):
    new_participant = Participant(participant_budget, json_poll)
    new_participant.setMatrices(matrices)
    new_participant.setEpsilons(epsilons)
    participants.append(new_participant)
    


In [9]:
#TODO: read weights from dist_accordion

weights = pollToPathWeights(json_poll)
ps = pollToPaths(json_poll)

ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
         zipped.append((path, weight[pathToKey(path)]))
    ans_dist_zip[question] = zipped

#participant.createAnswer(ans_dist_zip)

In [10]:
#TODO: set answer dist

#TODO: run RR

#TODO: compare to ground truth

#TODO: display graphs